# TODO: Zklend

In [6]:
from typing import Dict

import IPython.display
import pandas

from classes import State
import constants
import db

## Load and prepare events

In [7]:
# Establish the connection.
connection = db.establish_connection()

# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address='{constants.Protocol.ZKLEND.value}'
    AND
        key_name IN ('Deposit', 'Withdrawal', 'CollateralEnabled', 'CollateralDisabled', 'Borrowing', 'Repayment', 'Liquidation', 'AccumulatorsSync')
    ORDER BY
        block_number ASC, id;
    """,
    con = connection,
)

# Close the connection.
connection.close()

/tmp/ipykernel_111994/2710962983.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zklend_events = pandas.read_sql(


In [8]:
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [9]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

# TODO: Debugging negative deposits below

In [10]:
# TODO: negative deposits?
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min())

deposit ETH -300437786.967166206714
deposit wBTC 0
deposit USDC -1.39246404407114132779
deposit DAI 0
deposit USDT -4997676.927172805096183238170
borrowings ETH 0
borrowings wBTC 0
borrowings USDC 0
borrowings DAI 0
borrowings USDT 0


In [11]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]

[('0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1',
  Decimal('-4997676.927172805096183238170'))]

In [ ]:
# TODO: Get token prices here: https://www.coingecko.com/en/api/documentation